In [1]:
!pip install torch_scatter torcheeg torch_geometric -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.1/295.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 57.8 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.utils as utils
from torch.utils.data import DataLoader, Subset ,WeightedRandomSampler
from torcheeg.models import CCNN
from torcheeg import transforms
from torcheeg.transforms import ToGrid
from torcheeg.datasets import SEEDIVDataset,SEEDIVFeatureDataset
from torcheeg.datasets.constants import SEED_IV_CHANNEL_LOCATION_DICT
from torcheeg.transforms import ToG
from torcheeg.datasets.constants import SEED_IV_ADJACENCY_MATRIX
from torcheeg.models import DGCNN
import torch_geometric.loader as geom_loader # Special loader for graphs
import copy
import scipy.signal as signal
import random
import numpy as np
import os

In [3]:
# 1. Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Set a fixed random seed for reproducibility across different libraries.
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
set_seed(42)

In [5]:
dataset = SEEDIVFeatureDataset(
    io_path='./tmp_out/seed_iv_features',
    root_path='/kaggle/input/seed-iv/eeg_feature_smooth',
    feature=['de_LDS'], 
    num_worker=0,
    offline_transform=transforms.Compose([
        transforms.To2d(), 
        transforms.Lambda(lambda x: torch.tensor(x).float())]),
    label_transform=transforms.Select('emotion'),
   
)

[2025-12-11 10:44:49] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./tmp_out/seed_iv_features.
[2025-12-11 10:44:49] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/45 [00:00<?, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 1it [00:00,  4.73it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 17it [00:00, 65.72it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 35it [00:00, 106.55it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 54it [00:00, 132.54it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 73it [00:00, 150.85it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_2015111

In [6]:
# This DataFrame contains columns like: subject_id, trial_id, emotion, etc.
meta_info = dataset.info 
all_subjects = sorted(meta_info['subject_id'].unique())
original_labels = meta_info['emotion'].unique()

print(f"Original Labels in Data: {sorted(original_labels)}")
print(f"Subjects Found: {all_subjects}")

Original Labels in Data: [0, 1, 2, 3]
Subjects Found: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [7]:
# Parameters
EPOCHS = 50
BATCH_SIZE = 16
LR = 0.005
PATIENCE_LR = 3
REDUCE_FACTOR = 0.5
PATIENCE_ES = 15
WEIGHT_DECAY = 0.01
HIDE_CHANNELS = 32
LABEL_SMOOTHING = 0.25


In [8]:
# Subject-Dependent Loop

# Create a directory to save the best models
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

subject_results = {}

print("Start Subject-Dependent Evaluation...")

for subject_id in all_subjects:
    print(f"\n{'='*30} Processing Subject {subject_id} {'='*30}")
    
    # Split Data (Train/Test) by Trial ID 
    subject_df = meta_info[meta_info['subject_id'] == subject_id]
    all_trials = subject_df['trial_id'].unique().tolist()
    
    # Pick 5 random videos for testing 
    random.seed(42) 
    test_trials = random.sample(all_trials, 5)
    train_trials = [t for t in all_trials if t not in test_trials]
    
    # Get Indices
    train_indices = subject_df[subject_df['trial_id'].isin(train_trials)].index.tolist()
    test_indices = subject_df[subject_df['trial_id'].isin(test_trials)].index.tolist()
    
    # Create Subsets
    train_set = Subset(dataset, train_indices)
    test_set = Subset(dataset, test_indices)
    
    # Weighted Sampler (To balance emotions)
    y_train = meta_info.iloc[train_indices]['emotion'].values
    class_counts = np.bincount(y_train)
    class_weights = 1. / (class_counts + 1e-6)
    sample_weights = [class_weights[y] for y in y_train]
    
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
    
    # Loaders
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, sampler=sampler)
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)
    
    # --- 3. Initialize Model ---
    model = DGCNN(
        in_channels=5, 
        num_electrodes=62, 
        num_layers=2, 
        hid_channels=HIDE_CHANNELS, 
        num_classes=4,
    ).to(device)
    
    criterion = nn.CrossEntropyLoss( label_smoothing = LABEL_SMOOTHING)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=REDUCE_FACTOR, patience=PATIENCE_LR)
   
    # Training Loop 
    best_val_acc = 0.0
    counter = 0 # For early stopping
    
    for epoch in range(EPOCHS):
        model.train()
        correct = 0
        total = 0
        train_loss = 0
        
        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device).long()
            
            if len(X.shape) == 4: X = X.squeeze(1) 
          
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
            
       
        avg_train_loss = train_loss / len(train_loader)
        train_acc = 100 * correct / total
            
        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss = 0
        with torch.no_grad():
            for X, y in test_loader:
                X = X.to(device)
                y = y.to(device).long()
                if len(X.shape) == 4: X = X.squeeze(1)
                
                outputs = model(X)
                val_loss += criterion(outputs, y).item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += y.size(0)
                val_correct += (predicted == y).sum().item()
        
        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(test_loader)
        scheduler.step(avg_val_loss)

        print(f"Epoch {epoch+1:02d} | "
              f"Train Loss: {avg_train_loss:.4f} Acc: {train_acc:.2f}% | "
              f"Val Loss: {avg_val_loss:.4f} Acc: {val_acc:.2f}%")
        
        # Early Stopping Check
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            counter = 0

            save_path = f"saved_models/subject_{subject_id}_best.pth"
            torch.save(model.state_dict(), save_path)
            
        else:
            counter += 1
            if counter >= PATIENCE_ES:
                print(f"Early stopping at epoch {epoch}")
                break
                
    print(f"Subject {subject_id} Best Acc: {best_val_acc:.2f}%")
    subject_results[subject_id] = best_val_acc

# ================= FINAL REPORT =================
print("\n" + "="*40)
print("FINAL RESULTS REPORT ")
print("="*40)

all_accuracies = list(subject_results.values())

for sub_id in sorted(subject_results.keys()):
    print(f"Subject {sub_id}: {subject_results[sub_id]:.2f}%")

print(f"\nAverage Accuracy: {np.mean(all_accuracies):.2f}%")

if len(subject_results) > 0:
    best_sub_id = max(subject_results, key=subject_results.get)
    print(f"HIGHEST ACCURACY: Subject {best_sub_id} ({subject_results[best_sub_id]:.2f}%)")
print("="*40)

Start Subject-Dependent Evaluation...

============================== Processing Subject 1 ==============================
Epoch 01 | Train Loss: 1.1265 Acc: 70.07% | Val Loss: 1.2126 Acc: 56.80%
Epoch 02 | Train Loss: 0.8143 Acc: 95.56% | Val Loss: 1.2273 Acc: 71.14%
Epoch 03 | Train Loss: 0.7832 Acc: 97.71% | Val Loss: 1.1441 Acc: 69.49%
Epoch 04 | Train Loss: 0.7592 Acc: 98.88% | Val Loss: 1.4175 Acc: 48.16%
Epoch 05 | Train Loss: 0.7510 Acc: 99.24% | Val Loss: 1.1350 Acc: 80.70%
Epoch 06 | Train Loss: 0.7658 Acc: 97.71% | Val Loss: 1.2410 Acc: 55.15%
Epoch 07 | Train Loss: 0.7386 Acc: 99.54% | Val Loss: 1.2569 Acc: 61.95%
Epoch 08 | Train Loss: 0.7379 Acc: 99.59% | Val Loss: 1.3077 Acc: 51.47%
Epoch 09 | Train Loss: 0.7361 Acc: 99.75% | Val Loss: 1.2489 Acc: 52.21%
Epoch 10 | Train Loss: 0.7201 Acc: 99.90% | Val Loss: 1.2715 Acc: 59.93%
Epoch 11 | Train Loss: 0.7189 Acc: 100.00% | Val Loss: 1.3256 Acc: 55.15%
Epoch 12 | Train Loss: 0.7191 Acc: 100.00% | Val Loss: 1.1983 Acc: 61.21%
